In [ ]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    BsplineTrajectory,
    DiagramBuilder,
    KinematicTrajectoryOptimization,
    MeshcatVisualizer,
    RotationMatrix,
    MeshcatVisualizerParams,
    AbstractValue,
    RollPitchYaw,
    MinimumDistanceLowerBoundConstraint,
    Parser,
    PositionConstraint,
    Rgba,
    RigidTransform,
    Context,
    OutputPort,
    Simulator,
    Role,
    ConstantValueSource,
    Solve,
    LoadModelDirectivesFromString,
    ProcessModelDirectives,
    Sphere,
    LeafSystem,
    StartMeshcat,
    ConstantVectorSource,
    JacobianWrtVariable,
    IiwaControlMode,
    SimIiwaDriver
)

from manipulation.meshcat_utils import PublishPositionTrajectory
from manipulation.scenarios import AddIiwa, AddPlanarIiwa, AddWsg
from manipulation.utils import ConfigureParser
from manipulation.utils import RenderDiagram

from ShishKebot.TrajectoryPublisher import TrajectoryPublisher

In [ ]:
meshcat = StartMeshcat()

In [ ]:
model_directive = """
directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0.0]
        iiwa_joint_2: [0.6]
        iiwa_joint_3: [0.0]
        iiwa_joint_4: [-1.75]
        iiwa_joint_5: [0.0]
        iiwa_joint_6: [1.0]
        iiwa_joint_7: [0.0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://manipulation/schunk_wsg_50_welded_fingers.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
"""

In [ ]:
builder = DiagramBuilder()

# Add our iiwa to the scene
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
directives = LoadModelDirectivesFromString(model_directive)
parser = Parser(plant)
parser.package_map().Add("manipulation", "/home/trevor/.local/lib/python3.10/site-packages/manipulation/models/")
models = ProcessModelDirectives(directives, plant, parser)
iiwa = plant.GetModelInstanceByName("iiwa")
plant.Finalize()

# Connect meshcat to scene
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

trajectory = builder.AddNamedSystem("Trajectory", TrajectoryPublisher(plant))

controller = builder.AddSystem(SimIiwaDriver(
  control_mode=IiwaControlMode.kPositionAndTorque, 
  controller_plant=plant, 
  ext_joint_filter_tau=0.01,
  kp_gains=np.full(plant.num_positions(iiwa), 100))
)

zeros = builder.AddSystem(ConstantVectorSource(np.zeros(7)))

builder.Connect(plant.get_state_output_port(iiwa), controller.GetInputPort("state"))
builder.Connect(plant.get_generalized_contact_forces_output_port(iiwa), controller.GetInputPort("generalized_contact_forces"))
builder.Connect(zeros.get_output_port(0), controller.GetInputPort("torque"))

R = RotationMatrix(RollPitchYaw(0, 0, 45))
X_WGoal = RigidTransform(R, [0.4, 0.0, 0.5])
poses = builder.AddSystem(ConstantValueSource(AbstractValue.Make(X_WGoal)))
builder.Connect(poses.get_output_port(0), trajectory.GetInputPort("pose_desired"))
builder.Connect(controller.GetOutputPort("position_measured"), trajectory.GetInputPort("iiwa_position_measured"))
builder.Connect(trajectory.GetOutputPort("iiwa_position_cmd"), controller.GetInputPort("position"))

builder.Connect(controller.GetOutputPort("actuation"), plant.get_actuation_input_port(iiwa))

diagram = builder.Build()
simulator = Simulator(diagram)

In [ ]:
meshcat.StartRecording(set_visualizations_while_recording=False)
simulator.AdvanceTo(3)
meshcat.PublishRecording()